# On Multiple Linear Regression - Codealong

In [39]:
# Let's import numpy, pandas, and make sure our matplotlib will
# appear inline.



The main idea here is pretty simple. Whereas, in simple linear regression we took our dependent variable to be a function only of a single independent variable, here we'll be taking the dependent variable to be a function of multiple independent variables.

Our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

Is this still a best-fit *line*? Well, no. What does the graph of, say, z = x + y look like? [Here's](https://academo.org/demos/3d-surface-plotter/) a 3d-plotter. (Of course, once we get x's with subscripts beyond 2 it's going to be very hard to visualize. But in practice linear regressions can make use of dozens or even of hundreds of independent variables!)

I want to focus here more on what coding a multiple regression looks like in Python. But you might be wondering: Is it possible to calculate the betas by hand?

Yes! See [here](https://stattrek.com/multiple-regression/regression-coefficients.aspx) for a nice explanation and example.

## Dealing with Categorical Variables

One issue we'd like to resolve is what to do with categorical variables, i.e. variables that represent categories rather than continua. In a Pandas DataFrame, these columns may well have strings or objects for values, but they need not. A certain heart-disease dataset from Kaggle, for example, has a target variable that takes values 0-4, each representing a different stage of heart disease.

### Dummying

One very effective way of dealing with categorical variables is to dummy them out. What this involves is making a new column for _each categorical value in the column we're dummying out_.

These new columns will be filled only with 0's and 1's, a 1 representing the presence of the relevant categorical value.

Let's look at a simple example:

In [1]:
# Read in the comma_use dataset.



For more on this dataset see [here](https://fivethirtyeight.com/features/elitist-superfluous-or-popular-we-polled-americans-on-the-oxford-comma/).

In [2]:
# Let's look at the head.



In [3]:
# What's the shape of this dataframe?



In [4]:
# Let's drop all rows with any null values.



In [5]:
# What's the shape of our dataframe now?



In [54]:
# Let's try using sklearn's OneHotEncoder to create our dummy columns:

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first')
comma_trans = ohe.fit_transform(comma_use)

Could we have used ```pd.get_dummies()``` instead?

Well, yes. And in fact ```get_dummies()``` is in some ways easier; for one thing, it's built right into Pandas. But there are drawbacks with it as well. See the *bottom* of [this link](https://stackoverflow.com/questions/36631163/pandas-get-dummies-vs-sklearns-onehotencoder-what-are-the-pros-and-cons) for a good explanation.

So what did the encoder do?

In [30]:
#comma_trans.todense()

In [31]:
#ohe.get_feature_names()

In [32]:
# df = pd.DataFrame(comma_trans.todense(), columns=ohe.get_feature_names())
# df.head()

## Wine Dataset

In [6]:
# Read in the wine dataset



In [7]:
# Check out the head



In [8]:
# Any nulls? Any non-numerical values?
# Let's use .info()



## Model Selection

Let's imagine that I'm going to try to predict wine quality based on the other features.

Now: Which columns (predictors) should I choose? There are 12 predictors I could choose from For each of these predictors, I could either use it or not use it in my model, which means that there are 2^12 = 4096 different models I could construct! Well, okay, one of these is the "empty model" with no predictors in it. But there are still 4095 models from which I can choose.

How can I decide which predictors to use in my model?

### Correlation

In [9]:
# Use the .corr() DataFrame method to find out about the
# correlation values between all pairs of variables!



In [10]:
import seaborn as sns
sns.set(rc={'figure.figsize':(8, 8)})

# Use the .heatmap method to depict the relationships visually!


In [16]:
# Let's look at the correlations with 'quality'
# (our dependent variable) in particular.
# Since wine.corr()['quality'] is a Series,
# we can call .sort_values() on it.



In [15]:
# Let's choose alcohol and density as our predictors



## Multiple Regression in StatsModels

In [121]:
import statsmodels.api as sm

In [17]:
predictors = sm.add_constant(X)
model = sm.OLS(y, predictors).fit()
model.summary()

In [18]:
# Let's try a new model that also includes
# the volatile acidity.



In [19]:
predictors = sm.add_constant(X2)
model = sm.OLS(np.asarray(y), predictors).fit()
model.summary()

## Multiple Regression in Scikit-Learn

In [136]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [20]:
# Let's split our data into train and test sets.
# Let's also set a random state for reproducibility.



In [212]:
# Let's create a StandardScaler object to scale our data for us.


# Now we'll apply it to our data by using the .fit() and transform() methods.



In [21]:
# Now we can fit a LinearRegression object to our training data!



In [22]:
# And score it on our testing set



In [23]:
# We can use the .coef_ attribute to recover the results
# of the regression.



In [24]:
# And .intercept_



## Recursive Feature Elimination

The idea behind recursive feature elimination is to start with all predictive features and then build down to a small set of features slowly, by eliminating the features with the lowest coefficients.

That is:
1. Start with a model with _all_ $n$ predictors;
2. find the predictor with the smallest coefficient;
3. throw that predictor out and build a model with the remining $n-1$ predictors;
4. set $n = n-1$ and repeat until $n-1$ has the value you want!

### Recursive Feature Elimination in Scikit-Learn

In [25]:
from sklearn.feature_selection import RFE

lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=1)
select = select.fit(X = X_train_sc,
                    y = Y_train)

# select.support_

In [26]:
# select.ranking_

### Sklearn Metrics

The metrics module in sklearn has a number of metrics that we can use to meaure the accuracy of our model, including the $R^2$ score, the mean absolute error and the mean squared error. Note that the default 'score' on our model object is the $R^2$ score.

In [27]:
#metrics.r2_score(Y_test, lr.predict(X_test))

In [28]:
#metrics.mean_absolute_error(Y_test, lr.predict(X_test))

In [29]:
#metrics.mean_squared_error(Y_test, lr.predict(X_test))